In [ ]:
!gdown https://drive.google.com/uc?id=1Kj47wSBY-9k6Kzfy8r-zlT1H-vUaxgp8
!gdown https://drive.google.com/uc?id=1bQ3qw-5O06STYcpoOruKWEANJt2qYRCa
!gdown https://drive.google.com/uc?id=1Nh1XgETEAHVjxgpuk6qqxE72MYtZFmqC

In [ ]:
!unzip /root/Project/class/fire_detection/fire_yolo.zip -d dataset/

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import os
import random

def plot_images_with_boxes(image_paths, label_paths):
    fig, axes = plt.subplots(2, 3, figsize=(10, 10))
    axes = axes.ravel()  # 2x3 그리드를 1차원 배열로 변환

    for idx, (image_path, label_path) in enumerate(zip(image_paths, label_paths)):
        # 이미지 읽기
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR 포맷을 사용하므로 RGB로 변환

        # 이미지 크기 얻기
        height, width, _ = image.shape

        # 라벨 읽기
        with open(label_path, 'r') as f:
            labels = f.readlines()

        # 이미지 플롯 설정
        ax = axes[idx]
        ax.imshow(image)
        ax.axis('off')  # 축 숨기기

        for label in labels:
            parts = label.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1]) * width
            y_center = float(parts[2]) * height
            bbox_width = float(parts[3]) * width
            bbox_height = float(parts[4]) * height

            # 좌상단 좌표로 변환
            x_min = x_center - bbox_width / 2
            y_min = y_center - bbox_height / 2

            # 박스 그리기
            rect = patches.Rectangle((x_min, y_min), bbox_width, bbox_height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

    plt.tight_layout()
    plt.show()

def get_random_image_label_paths(image_folder, label_folder, num_samples=6):
    all_images = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    sampled_images = random.sample(all_images, num_samples)
    
    image_paths = [os.path.join(image_folder, img) for img in sampled_images]
    label_paths = [os.path.join(label_folder, os.path.splitext(img)[0] + '.txt') for img in sampled_images]

    return image_paths, label_paths

# 이미지와 라벨 파일이 있는 폴더 경로
image_folder = '/root/Project/class/fire_detection/dataset/fire/train/images'
label_folder = '/root/Project/class/fire_detection/dataset/fire/train/labels'

# 랜덤으로 이미지와 라벨 파일 경로 추출
image_paths, label_paths = get_random_image_label_paths(image_folder, label_folder)

# 함수 호출
plot_images_with_boxes(image_paths, label_paths)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11x.pt") 
# pretrained 모델명 기재
# yolov8은 yolov8n.pt, yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
# 모델을 프리트레인으로 제공 오른쪽으로 갈수록 무거운 모델

model.train(
    data="fire_data.yaml",
    project="Yolov11",
    name="firedetection",
    epochs=100, 
    imgsz=640, 
    batch=8,
    )
# 기타 파라메터는 https://docs.ultralytics.com/ko/ 여기서 확인
# 파라메터 설정하지 않은 값은 default로 실행됨

In [ ]:
!tensorboard --logdir=Yolov11/firedetection

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("/root/Project/class/fire_detection/Yolov11/firedetection4/weights/best.pt")

# Open the video file
video_path = "/root/Project/class/fire_detection/test_video.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/root/Project/class/fire_detection/annotated_video.mp4', fourcc, fps, (width, height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame, conf=0.5, iou=0.8)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Write the annotated frame to the output video
        out.write(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("/root/Project/class/fire_detection/Yolov11/firedetection4/weights/best.pt")

# Open the video file
video_path = "/root/Project/class/fire_detection/test_video.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/root/Project/class/fire_detection/annotated_video_h.mp4', fourcc, fps, (width, height))

# Define a function for color histogram analysis
def is_fire_like(frame, threshold=0.008):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the red color range
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = mask1 | mask2

    # Apply the red mask to get red areas
    red_only = cv2.bitwise_and(frame, frame, mask=red_mask)

    # Calculate histogram and normalize
    hist = cv2.calcHist([red_only], [0], red_mask, [256], [0, 256])
    hist = hist / hist.sum()

    # Measure the irregularity in the red distribution
    irregularity = np.std(hist)

    return irregularity > threshold

# Loop through the video frames
while cap.isOpened():
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Process each detection result
        for result in results[0].boxes:
            # Extract the bounding box coordinates
            x1, y1, x2, y2 = map(int, result.xyxy[0])
            cropped_frame = frame[y1:y2, x1:x2]

            # Apply color histogram analysis to filter out false positives
            if is_fire_like(cropped_frame):
                # Draw the bounding box if it passes the color analysis
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, "Fire", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Write the annotated frame to the output video
        out.write(frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()
